# Data Preparation
- convert json file to csv file
- split transcript to transaction and the rest.
- convert categorical feature to numeric

In [1]:
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
def prepare_transations(transcript, outfile="data/transactions.csv"):
    """
        get transactions from transcript and save to outfile
    """
    transactions = transcript[transcript.event == 'transaction']
    transactions['amount'] = transactions.value.apply(lambda x: x['amount'])
    transactions = transactions[["person", "amount", "time"]]
    transactions.to_csv(outfile)
    
    return transactions
    
prepare_transations(transcript)

/Users/hg_kim/.pyenv/versions/3.6.10/envs/drlnd/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,person,amount,time
12654,02c083884c7d45b39cc68e1314fec56c,0.83,0
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,34.56,0
12659,54890f68699049c2a04d415abc25e717,13.23,0
12670,b2f1cd155b864803ad8334cdf13c4bd2,19.51,0
12671,fe97aa22dd3e48c8b143116a8403dd52,18.97,0
...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,1.59,714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,9.53,714
306531,a00058cf10334a308c68e7631c529907,3.61,714
306532,76ddbd6576844afe811f1a3c0fbb5bec,3.53,714


In [3]:
def prepare_offers(transcript, outfile="data/offers.csv"):
    """
        get offer received, offer viewed, offer completed from transcript 
        also insert reward column for offer completed and save to outfile
    """
    offer_types = ['offer received', 'offer viewed']
    offers = transcript[transcript.event.isin(offer_types)][['person', 'event', 'value', 'time']]
    offers.value = offers.value.apply(lambda x: x['offer id'])
    offers.columns = ['person', 'event', 'offer_id', 'time']

    offer_completed = transcript[transcript.event == 'offer completed'][['person', 'event', 'value', 'time']]
    offer_completed['offer_id'] = offer_completed.value.apply(lambda x: x['offer_id']).to_list()
    offer_completed['reward'] = offer_completed.value.apply(lambda x: x['reward']).to_list()
    offer_completed = offer_completed[['person', 'event', 'offer_id', 'reward', 'time']]

    total_offers = pd.concat([offers, offer_completed], axis=0)
    total_offers.to_csv(outfile)
    return total_offers

prepare_offers(transcript)

,person,event,offer_id,time,reward
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,2906b810c7d4411798c6938adc9daaa5,0,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,fafdcd668e3743c1bb461111dcafc2a4,0,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,4d5c57ea9a6940dd891ad53e9dbe8da0,0,NaN
...,...,...,...,...,...
306475,0c027f5f34dd4b9eba0a25785c611273,offer completed,2298d6c36e964ae4a3e7e9706d1fb8c2,714,3.0
306497,a6f84f4e976f44508c358cc9aba6d2b3,offer completed,2298d6c36e964ae4a3e7e9706d1fb8c2,714,3.0
306506,b895c57e8cd047a8872ce02aa54759d6,offer completed,fafdcd668e3743c1bb461111dcafc2a4,714,2.0
306509,8431c16f8e1d440880db371a68f82dd0,offer completed,fafdcd668e3743c1bb461111dcafc2a4,714,2.0


In [6]:
def prepare_profile(profile, outfile="data/profile.csv"):
    """
        save profile to csv with handling na value.
        na value is replaced to mean value and
        new columns are inserted to remark na (incom_na, age_na)
    """
    profile.index = profile.id
    profile = profile.drop(columns=['id'])
    
    profile['not_na'] = 1 - profile.income.isna().astype(int)
    profile.income = profile.income.fillna(profile.income.mean())
    profile = pd.get_dummies(profile)
    
    profile.age = profile.age.replace({118: np.NaN})
    profile.age = profile.age.fillna(profile.age.mean())
    
    profile['member_since'] = profile.became_member_on.apply(lambda x: str(x)[:4]).astype(int)
    profile = profile.drop(columns='became_member_on')
    profile.to_csv(outfile)
    return profile

prepare_profile(profile.copy())

,age,income,not_na,gender_F,gender_M,gender_O,member_since
id,,,,,,,
68be06ca386d4c31939f3a4f0e3dd783,54.393524,65404.991568,0,0,0,0,2017
0610b486422d4921ae7d2bf64640c50b,55.000000,112000.000000,1,1,0,0,2017
38fe809add3b4fcf9315a9694bb96ff5,54.393524,65404.991568,0,0,0,0,2018
78afa995795e4d85b5d9ceeca43f5fef,75.000000,100000.000000,1,1,0,0,2017
a03223e636434f42ac4c3df47e8bac43,54.393524,65404.991568,0,0,0,0,2017
...,...,...,...,...,...,...,...
6d5f3a774f3d4714ab0c092238f3a1d7,45.000000,54000.000000,1,1,0,0,2018
2cb4f97358b841b9a9773a7aa05a9d77,61.000000,72000.000000,1,0,1,0,2018
01d26f638c274aa0b965d24cefe3183f,49.000000,73000.000000,1,0,1,0,2017


In [26]:

def prepare_portfolio(portfolio, outfile='data/portfolio-dummy.csv'):
    """
        change portfolio for model training
        convert categorical data to 1 value
    """
    
    portfolio_pivot = portfolio.explode('channels') \
        .pivot_table(index=["id", "reward", "difficulty", "duration", "offer_type"], columns=["channels"], aggfunc="size") \
        .fillna(0)
    portfolio_pivot = portfolio_pivot.reset_index()
    portfolio_pivot.index = portfolio_pivot.id
    portfolio_pivot = portfolio_pivot.drop(columns='id')
    portfolio_dummies = pd.get_dummies(portfolio_pivot)
    
    portfolio_pivot.columns.name = ""
    portfolio_dummies.to_csv(outfile)
    
    return portfolio_dummies
    
prepare_portfolio(portfolio)

,reward,difficulty,duration,email,mobile,social,web,offer_type_bogo,offer_type_discount,offer_type_informational
id,,,,,,,,,,
0b1e1539f2cc45b7b9fa7c272da2e1d7,5,20,10,1.0,0.0,0.0,1.0,0,1,0
2298d6c36e964ae4a3e7e9706d1fb8c2,3,7,7,1.0,1.0,1.0,1.0,0,1,0
2906b810c7d4411798c6938adc9daaa5,2,10,7,1.0,1.0,0.0,1.0,0,1,0
3f207df678b143eea3cee63160fa8bed,0,0,4,1.0,1.0,0.0,1.0,0,0,1
4d5c57ea9a6940dd891ad53e9dbe8da0,10,10,5,1.0,1.0,1.0,1.0,1,0,0
5a8bc65990b245e5a138643cd4eb9837,0,0,3,1.0,1.0,1.0,0.0,0,0,1
9b98b8c7a33c4b65b9aebfe6a799e6d9,5,5,7,1.0,1.0,0.0,1.0,1,0,0
ae264e3637204a6fb9bb56bc8210ddfd,10,10,7,1.0,1.0,1.0,0.0,1,0,0
f19421c1d4aa40978ebb69ca19b0e20d,5,5,5,1.0,1.0,1.0,1.0,1,0,0
